### 추세
- 추세라는 것은 시장의 현재 방향성을 주관적 정의에 따라 구분한 것으로 절대적이지 않음
- 나는 추세를 1)상승추세 2)횡보추세 3)하락추세 세가지로 분류함
- 각 추세는 기본적으로 아래의 기준으로 정의함
#### 1. 상승추세
- 대충 봤을때 누가 보더라도 상승추세라고 말할 수 있는 패턴
- 주어진 타임 프레임에서 종가 > 시가 이어야 함
- 양의 기울기를 가진 채널을 따라 저점과 고점을 높혀가는 패턴은 베스트
- 횡보 -> 급등 -> 횡보 의 경우 급등구간이 처음 2/3 이전에 발생해야 함

#### 2. 하락추세
- 대충 봤을때 누가 보더라도 하락추세라고 말할 수 있는 패턴
- 주어진 타임 프레임에서 종가 < 시가 이어야 함
- 음의 기울기를 가진 채널을 따라 저점과 고점을 낮춰가는 패턴은 베스트
- 횡보후 마지막 1/3 구간에서 급락하는 경우도 하락 추세로 분류 

#### 3. 횡보추세
- 상승추세와 하락추세가 아니면 횡보

In [1]:
import sys, os
sys.path.append('../..')
from tools import ML
from tools.instruments import instruments
from tools.display import ohlc_chart, highchart
from tools.datatools import norm
import h5py
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time

In [2]:
%load_ext autoreload
%aimport tools.ML
%aimport tools.display
%autoreload 2

### 0. Parameters

In [3]:
period= 200
filepath = os.path.join('..','..','data','trend index','training set', f'trend_{period}day.hdf')
training_period = '2009-12-31'

### 1. Gathering Data

In [8]:
quotes = instruments.quotes(fields='ohlc')[:training_period].copy()
symbols = list(quotes.columns.levels[0])

In [ ]:
file = h5py.File(filepath, mode='a')

num_dim = 4 #open, high, low, close
if 'ohlc' not in file.keys():
        file.create_dataset('ohlc', (0, num_dim, period), maxshape=(None, num_dim, period), dtype='float32', compression='gzip')
        file.create_dataset('trend', (0,), maxshape=(None,), dtype='i')
        
Train, Trend  = file['ohlc'], file['trend']

size = len(Train)
while True:
    # 상품 랜덤 선택
    symbol = np.random.choice(symbols)
    name = instruments[symbol].name
    quote = quotes[symbol].dropna()
    #print(instruments[symbol].name)
    
    # 10세트 데이터 랜덤 선택. quote 없으면 넘어감
    if len(quote) > period+100:
        indexes = np.random.randint(0, len(quote)-period, 10)
    else:
        continue
        
    for index in indexes:
        # 데이터 셋 선택 및 규격확
        data = quote.iloc[index:index+period]
        ohlc = norm(data.values)#(data - data.mean())/data.std().mean()
        highchart([{ 'data':data, 'name':name, 'type':'candlestick'}], height=600)
        
        time.sleep(0.1)
        print(f"data length: {size}")
        trend = input("추세? 하락(-1), 없음(0), 상승(1): ")
        if (trend not in ['-1','0','1']):
            print(f"You pressed trend:{trend}")
            break
        else:
            size = Train.shape[0]
            
            Train.resize(size+1, axis=0)
            Trend.resize(size+1, axis=0)
            #sktime 에 사용되는 데이터 구조가 (데이터 갯수, num_dims, period) 라서 변경함
            Train[size] = np.moveaxis(ohlc, 0, 1) 
            Trend[size] = int(trend)

        clear_output(wait=True)
    
    if (trend not in ['-1','0','1']):
            break
file.close()

data length: 1002


In [47]:
file.close()

### 2. Verification

In [48]:
from collections import Counter

In [49]:
file = h5py.File(filepath, mode='r')

In [50]:
print(file['ohlc'].shape)
print(file['trend'].shape)

(1006, 4, 50)
(1006,)


In [51]:
Y = file['trend']
Counter(Y)

Counter({1: 293, -1: 259, 0: 454})

In [52]:
file.close()